Import Libraries

In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

Read the table

In [4]:
wiki = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(wiki,'xml')
table = soup.find('table',{'class':'wikitable sortable'})
rows = table.find_all('tr')


Extract raw tables

In [5]:
frame = []
for row in rows:
    frame.append([t.text.strip() for t in row.find_all('td')])


Create a dataframe

In [6]:
df = pd.DataFrame(frame, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['Borough'].isnull()]  

Clean not assigned rows of column Borough

In [7]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True) 
new_df = df.reset_index()

Group by postal code

In [8]:
grouped_df= new_df.groupby('PostalCode').agg(lambda x: ','.join(x))

Fix the repeating Boroughs

In [9]:
grouped_df.loc[grouped_df['Neighbourhood']=="Not assigned",'Neighbourhood']=grouped_df.loc[grouped_df['Neighbourhood']=="Not assigned",'Borough']
new_grouped_df = grouped_df.reset_index()
new_grouped_df['Borough']= new_grouped_df['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")


Shape

In [10]:
new_grouped_df.shape

(103, 3)

In [11]:
new_grouped_df

PostalCode          Borough  \
0          M1B      Scarborough   
1          M1C      Scarborough   
2          M1E      Scarborough   
3          M1G      Scarborough   
4          M1H      Scarborough   
5          M1J      Scarborough   
6          M1K      Scarborough   
7          M1L      Scarborough   
8          M1M      Scarborough   
9          M1N      Scarborough   
10         M1P      Scarborough   
11         M1R      Scarborough   
12         M1S      Scarborough   
13         M1T      Scarborough   
14         M1V      Scarborough   
15         M1W      Scarborough   
16         M1X      Scarborough   
17         M2H        NorthYork   
18         M2J        NorthYork   
19         M2K        NorthYork   
20         M2L        NorthYork   
21         M2M        NorthYork   
22         M2N        NorthYork   
23         M2P        NorthYork   
24         M2R        NorthYork   
25         M3A        NorthYork   
26         M3B        NorthYork   
27         M3C        NorthYork   
28         M3H        NorthYork   
29         M3J        NorthYork   
..         ...              ...   
73         M6C             York   
74         M6E             York   
75         M6G  DowntownToronto   
76         M6H      WestToronto   
77         M6J      WestToronto   
78         M6K      WestToronto   
79         M6L        NorthYork   
80         M6M             York   
81         M6N             York   
82         M6P      WestToronto   
83         M6R      WestToronto   
84         M6S      WestToronto   
85         M7A  DowntownToronto   
86         M7R      Mississauga   
87         M7Y      EastToronto   
88         M8V        Etobicoke   
89         M8W        Etobicoke   
90         M8X        Etobicoke   
91         M8Y        Etobicoke   
92         M8Z        Etobicoke   
93         M9A      Queen'sPark   
94         M9B        Etobicoke   
95         M9C        Etobicoke   
96         M9L        NorthYork   
97         M9M        NorthYork   
98         M9N             York   
99         M9P        Etobicoke   
100        M9R        Etobicoke   
101        M9V        Etobicoke   
102        M9W        Etobicoke   

                                         Neighbourhood  
0                                        Rouge,Malvern  
1                 Highland Creek,Rouge Hill,Port Union  
2                      Guildwood,Morningside,West Hill  
3                                               Woburn  
4                                            Cedarbrae  
5                                  Scarborough Village  
6            East Birchmount Park,Ionview,Kennedy Park  
7                        Clairlea,Golden Mile,Oakridge  
8        Cliffcrest,Cliffside,Scarborough Village West  
9                           Birch Cliff,Cliffside West  
10   Dorset Park,Scarborough Town Centre,Wexford He...  
11                                    Maryvale,Wexford  
12                                           Agincourt  
13               Clarks Corners,Sullivan,Tam O'Shanter  
14   Agincourt North,L'Amoreaux East,Milliken,Steel...  
15                                     L'Amoreaux West  
16                                         Upper Rouge  
17                                   Hillcrest Village  
18                          Fairview,Henry Farm,Oriole  
19                                     Bayview Village  
20                             Silver Hills,York Mills  
21                              Newtonbrook,Willowdale  
22                                    Willowdale South  
23                                     York Mills West  
24                                     Willowdale West  
25                                           Parkwoods  
26                                     Don Mills North  
27                     Flemingdon Park,Don Mills South  
28       Bathurst Manor,Downsview North,Wilson Heights  
29                      Northwood Park,York University  
..                                                 ...  
73       

# Second part of the week 3

Import CSV file

In [16]:
latlng_df = pd.read_csv('http://cocl.us/Geospatial_data')
latlng_df.columns = ['PostalCode', 'Latitude', 'Longitude']

PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476

Inner join with previous data frame

In [19]:
joined_df = pd.merge(new_grouped_df, latlng_df, on=['PostalCode'], how='inner')
joined_df

PostalCode          Borough  \
0          M1B      Scarborough   
1          M1C      Scarborough   
2          M1E      Scarborough   
3          M1G      Scarborough   
4          M1H      Scarborough   
5          M1J      Scarborough   
6          M1K      Scarborough   
7          M1L      Scarborough   
8          M1M      Scarborough   
9          M1N      Scarborough   
10         M1P      Scarborough   
11         M1R      Scarborough   
12         M1S      Scarborough   
13         M1T      Scarborough   
14         M1V      Scarborough   
15         M1W      Scarborough   
16         M1X      Scarborough   
17         M2H        NorthYork   
18         M2J        NorthYork   
19         M2K        NorthYork   
20         M2L        NorthYork   
21         M2M        NorthYork   
22         M2N        NorthYork   
23         M2P        NorthYork   
24         M2R        NorthYork   
25         M3A        NorthYork   
26         M3B        NorthYork   
27         M3C        NorthYork   
28         M3H        NorthYork   
29         M3J        NorthYork   
..         ...              ...   
73         M6C             York   
74         M6E             York   
75         M6G  DowntownToronto   
76         M6H      WestToronto   
77         M6J      WestToronto   
78         M6K      WestToronto   
79         M6L        NorthYork   
80         M6M             York   
81         M6N             York   
82         M6P      WestToronto   
83         M6R      WestToronto   
84         M6S      WestToronto   
85         M7A  DowntownToronto   
86         M7R      Mississauga   
87         M7Y      EastToronto   
88         M8V        Etobicoke   
89         M8W        Etobicoke   
90         M8X        Etobicoke   
91         M8Y        Etobicoke   
92         M8Z        Etobicoke   
93         M9A      Queen'sPark   
94         M9B        Etobicoke   
95         M9C        Etobicoke   
96         M9L        NorthYork   
97         M9M        NorthYork   
98         M9N             York   
99         M9P        Etobicoke   
100        M9R        Etobicoke   
101        M9V        Etobicoke   
102        M9W        Etobicoke   

                                         Neighbourhood   Latitude  Longitude  
0                                        Rouge,Malvern  43.806686 -79.194353  
1                 Highland Creek,Rouge Hill,Port Union  43.784535 -79.160497  
2                      Guildwood,Morningside,West Hill  43.763573 -79.188711  
3                                               Woburn  43.770992 -79.216917  
4                                            Cedarbrae  43.773136 -79.239476  
5                                  Scarborough Village  43.744734 -79.239476  
6            East Birchmount Park,Ionview,Kennedy Park  43.727929 -79.262029  
7                        Clairlea,Golden Mile,Oakridge  43.711112 -79.284577  
8        Cliffcrest,Cliffside,Scarborough Village West  43.716316 -79.239476  
9                           Birch Cliff,Cliffside West  43.692657 -79.264848  
10   Dorset Park,Scarborough Town Centre,Wexford He...  43.757410 -79.273304  
11                                    Maryvale,Wexford  43.750072 -79.295849  
12                                           Agincourt  43.794200 -79.262029  
13               Clarks Corners,Sullivan,Tam O'Shanter  43.781638 -79.304302  
14   Agincourt North,L'Amoreaux East,Milliken,Steel...  43.815252 -79.284577  
15                                     L'Amoreaux West  43.799525 -79.318389  
16                                         Upper Rouge  43.836125 -79.205636  
17                                   Hillcrest Village  43.803762 -79.363452  
18                          Fairview,Henry Farm,Oriole  43.778517 -79.346556  
19                                     Bayview Village  43.786947 -79.385975  
20                             Silver Hills,York Mills  43.757490 -79.374714  
21                              Newtonbrook,Willowdale  43.789053 -79.408493  
22              